# Assignment 4<br>

## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [wikipedia_data.html](wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [nfl.csv](nfl.csv)), MLB (baseball, in [mlb.csv](mlb.csv)), NBA (basketball, in [nba.csv](nba.csv)), or NHL (hockey, in [nhl.csv](nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!  

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), you are going to send in two ordered lists of values, the populations from the [wikipedia_data.html](wikipedia_data.html) file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%). You should only use data **from year 2018** for your analysis -- this is important!<br>
<br>
## Notes
1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of the grade for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the `assert` statements test, remember to collapse multiple teams in one city into a single value!

## Question 1<br>

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html('wikipedia_data.html')[1]
cities = cities.iloc[:-1, [0,3,5,6,7,8]]

#cities.head(20)

In [2]:
def clean_cities_df(cities_df, league):
    cities_df.rename(columns={'Population (2016 est.)[8]':'2016 Population'}, inplace=True)
    cities_df.replace(b'\xe2\x80\x94'.decode('utf-8'), '-', inplace=True)
    cities_df.replace('\[.*\]', '-', regex=True, inplace=True)
    
    cities_df = cities_df[['Metropolitan area', '2016 Population', league]]
    cities_df = cities_df[cities_df[league] != '-'] 
    if league == 'NFL':
        cities_df = cities_df[cities_df['Metropolitan area'] != 'Toronto']
        
    return cities_df


def parse_wiki_teams(group, league):
    # teams = re.findall('[A-Z\d]+[a-z]+(?=[A-Z])|[A-Z][a-z]+ [A-Z][a-z]+|[A-Z\d]+[a-z]+', group[league].iloc[0])  ## This worked for an older version of (I believe) pd.read_html
                                                                                                                 ## which read in the team names slightly differently.
    two_word_teams = ['White Sox',
                      'Red Sox',
                      'Blue Jays',
                      'Trail Blazers',
                      'Red Wings',
                      'Maple Leafs',
                      'Blue Jackets',
                      'Golden Knights']
    
    teams = re.findall('|'.join(two_word_teams)+'|[A-Z\d]+[a-z]+', group[league].iloc[0])
    
    del(group[league])
    modified_group = pd.concat([group]*len(teams))
    modified_group['team'] = teams
    
    return modified_group


def merge_and_agg(cities_and_teams, league_df, league):
    merged_df = pd.merge(cities_and_teams, league_df, how='inner', on='team')
    merged_df.set_index(['Metropolitan area', 'team'], inplace=True)
    merged_df = merged_df.astype(float)
    
    if league == 'NHL':
        merged_df['W/L%'] = merged_df['W'] / (merged_df['W'] + merged_df['L'])
    
    agg_df = merged_df.groupby('Metropolitan area').agg({'2016 Population':np.mean, 'W/L%':np.mean})
    agg_df['2016 Population'] = agg_df['2016 Population'].astype(int)
    
    return merged_df, agg_df, agg_df['2016 Population'], agg_df['W/L%']

In [3]:
nhl_df = pd.read_csv('nhl.csv')

#nhl_df.head(20)

In [4]:
def nhl_correlation(cities, nhl_df): 
    
    sports_league = 'NHL'
    
    cities_cleaned = clean_cities_df(cities, sports_league)
    
    cities_reorg   = cities_cleaned.groupby('Metropolitan area').apply(lambda x: parse_wiki_teams(x, sports_league)) 

    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df['team'] = nhl_df['team'].replace('\*', '', regex=True)
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.split()[-1])
    del(nhl_df['League'])
    
    for team in cities_reorg['team']:
        index = nhl_df.index[nhl_df['team'] == team.split()[-1]]
        nhl_df.loc[index, 'team'] = team
        
    merged_df, agg_df, population_by_region, win_loss_by_region = merge_and_agg(cities_reorg, nhl_df, sports_league)
    
    pearson_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return pearson_corr, cities_reorg, nhl_df, merged_df, agg_df

pearson_nhl, cities_reorg_nhl, nhl_df, merged_nhl_df, agg_nhl_df = nhl_correlation(cities, nhl_df)

print(pearson_nhl)
print(len(agg_nhl_df))
#agg_nhl_df

PearsonRResult(statistic=0.012486162921209881, pvalue=0.9497182859911808)
28


/tmp/ipykernel_16008/3903574712.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl_df['team'] = nhl_df['team'].replace('\*', '', regex=True)
/tmp/ipykernel_16008/3903574712.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl_df['team'] = nhl_df['team'].apply(lambda x: x.split()[-1])


In [ ]:
#cities_reorg_nhl

In [ ]:
#nhl_df

In [ ]:
#merged_nhl_df

## Question 2<br>

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
nba_df = pd.read_csv('nba.csv')

#nba_df.head(20)

In [6]:
def nba_correlation(cities, nba_df):

    sports_league = 'NBA'
    
    cities_cleaned = clean_cities_df(cities, sports_league)
    
    cities_reorg   = cities_cleaned.groupby('Metropolitan area').apply(lambda x: parse_wiki_teams(x, sports_league)) 
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].apply(lambda x: re.match('[\w\s]+', x).group()) #Searches beginning of string. .search looks for a match anywhere in the string
    nba_df['team'] = nba_df['team'].apply(lambda x: x.strip().split()[-1])
    nba_df.replace(b'\xe2\x80\x94'.decode('utf-8'), np.nan, inplace=True)
    del(nba_df['League'])
    
    for team in cities_reorg['team']:
        index = nba_df.index[nba_df['team'] == team.split()[-1]]
        nba_df.loc[index, 'team'] = team
        
    merged_df, agg_df, population_by_region, win_loss_by_region = merge_and_agg(cities_reorg, nba_df, sports_league)
    
    pearson_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return pearson_corr, cities_reorg, nba_df, merged_df, agg_df

pearson_nba, cities_reorg_nba, nba_df, merged_nba_df, agg_nba_df = nba_correlation(cities, nba_df)

print(pearson_nba)
print(len(agg_nba_df))
#agg_nba_df

PearsonRResult(statistic=-0.17636350642182935, pvalue=0.36932106185547353)
28


/tmp/ipykernel_16008/1083360393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].apply(lambda x: re.match('[\w\s]+', x).group()) #Searches beginning of string. .search looks for a match anywhere in the string
/tmp/ipykernel_16008/1083360393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].apply(lambda x: x.strip().split()[-1])
/tmp/ipykernel_16008/1083360393.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
#cities_reorg_nba

In [ ]:
#nba_df

In [ ]:
#merged_nba_df

## Question 3<br>

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [7]:
mlb_df = pd.read_csv('mlb.csv')

#mlb_df.head(20)

In [8]:
def mlb_correlation(cities, mlb_df):
    
    def split_team_name(team):
        if team == 'Boston Red Sox':
            return 'Red Sox'
        elif team == 'Chicago White Sox':
            return 'White Sox'
        else:
            return team.split()[-1]
    
    sports_league = 'MLB'
    
    cities_cleaned = clean_cities_df(cities, sports_league)
    
    cities_reorg   = cities_cleaned.groupby('Metropolitan area').apply(lambda x: parse_wiki_teams(x, sports_league))
    
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df.rename(columns={'W-L%':'W/L%'}, inplace=True)  ##To maintain consistency in W/L column name
    mlb_df.replace('--', np.nan, inplace=True)
    mlb_df['team'] = mlb_df['team'].apply(lambda x: split_team_name(x))
    del(mlb_df['League'])
    
    for team in cities_reorg['team']:
        if (team != 'Red Sox') and (team != 'White Sox'):
            index = mlb_df.index[mlb_df['team'] == team.split()[-1]]
            mlb_df.loc[index, 'team'] = team
            
    merged_df, agg_df, population_by_region, win_loss_by_region = merge_and_agg(cities_reorg, mlb_df, sports_league)
    
    pearson_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return pearson_corr, cities_reorg, mlb_df, merged_df, agg_df

pearson_mlb, cities_reorg_mlb, mlb_df, merged_mlb_df, agg_mlb_df = mlb_correlation(cities, mlb_df)

print(pearson_mlb)
print(len(agg_mlb_df))
#agg_mlb_df

PearsonRResult(statistic=0.15003737475409495, pvalue=0.46442827201123316)
26


/tmp/ipykernel_16008/2591440936.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlb_df.rename(columns={'W-L%':'W/L%'}, inplace=True)  ##To maintain consistency in W/L column name
/tmp/ipykernel_16008/2591440936.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlb_df.replace('--', np.nan, inplace=True)
/tmp/ipykernel_16008/2591440936.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [ ]:
#cities_reorg_mlb

In [ ]:
#mlb_df

In [ ]:
#merged_mlb_df

## Question 4<br>

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [9]:
nfl_df = pd.read_csv('nfl.csv')

#nfl_df.head(20)

In [10]:
def nfl_correlation(cities, nfl_df):
    
    sports_league = 'NFL'
    
    cities_cleaned = clean_cities_df(cities, sports_league)
    
    cities_reorg   = cities_cleaned.groupby('Metropolitan area').apply(lambda x: parse_wiki_teams(x, sports_league))
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df['team'] = nfl_df['team'].replace('[\*\+]', '', regex=True)
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split()[-1])
    nfl_df.rename(columns={'W-L%':'W/L%'}, inplace=True) ##To maintain consistency in W/L column name
    del(nfl_df['League'])
    
    merged_df, agg_df, population_by_region, win_loss_by_region = merge_and_agg(cities_reorg, nfl_df, sports_league)
    
    pearson_corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return pearson_corr, cities_reorg, nfl_df, merged_df, agg_df

pearson_nfl, cities_reorg_nfl, nfl_df, merged_nfl_df, agg_nfl_df = nfl_correlation(cities, nfl_df)

print(pearson_nfl)
print(len(agg_nfl_df))
#agg_nfl_df

PearsonRResult(statistic=0.004282141436393022, pvalue=0.9824114740736539)
29


/tmp/ipykernel_16008/3877692180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].replace('[\*\+]', '', regex=True)
/tmp/ipykernel_16008/3877692180.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split()[-1])
/tmp/ipykernel_16008/3877692180.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [ ]:
#cities_reorg_nfl

In [ ]:
#nfl_df

In [ ]:
#merged_nfl_df

## Question 5<br>

In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [11]:
def sports_team_performance(agg_nfl, agg_nba, agg_nhl, agg_mlb):
    agg_df_by_sport = {'NFL':agg_nfl, 'NBA':agg_nba, 'NHL':agg_nhl, 'MLB':agg_mlb}
    pairs_of_sports = ['NFL-NBA', 'NFL-NHL', 'NFL-MLB', 'NBA-NHL', 'NBA-MLB', 'NHL-MLB']
    p_values = pd.DataFrame({k:np.nan for k in ['NFL', 'NBA', 'NHL', 'MLB']}, index=['NFL', 'NBA', 'NHL', 'MLB'])
    
    for pair in pairs_of_sports:
        sport1 = pair.split('-')[0]
        sport2 = pair.split('-')[1]
        common_cities = pd.merge(agg_df_by_sport[sport1], agg_df_by_sport[sport2], how='inner', left_index=True, right_index=True, suffixes=['_'+sport1, '_'+sport2])
        ttest = stats.ttest_rel(common_cities['W/L%_'+sport1], common_cities['W/L%_'+sport2])
        p_values.loc[sport1, sport2] = ttest.pvalue
        p_values.loc[sport2, sport1] = ttest.pvalue
        
    return p_values

pvalues = sports_team_performance(agg_nfl_df, agg_nba_df, agg_nhl_df, agg_mlb_df)
print(pvalues, '\n')
pvalues
    

          NFL       NBA       NHL       MLB
NFL       NaN  0.937509  0.030318  0.803459
NBA  0.937509       NaN  0.022386  0.949566
NHL  0.030318  0.022386       NaN  0.000703
MLB  0.803459  0.949566  0.000703       NaN 



,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
